# Orquestador de ventas 360: resumen

## Input
1. Excel de demanda desagregada: con esta información se genera lista de claves y máximos. 
## Output
1. PDF's con nombre estructurado en folder específico. 
2. Base de datos con información de los contratos y convenios modificatorios generados con el string. 

## Notas
Diseño esto para, partiendo de la demanda desagregada: 
- Capturar contratos
- Descargar xlsx de sistemas SAI, CAMUNDA, SAGI, ZOHO y PISP. 
- Consultar exceles internos
- Organizar las órdenes descargadas
- Generar los pickle de las órdenes
Todo lo anterior es la información mínima indispensable para poder generar información de ventas, que será la referencia interna sobre la que desplegaremos el alimentado de reportes y la generación de reglas para dar retro al usuario.

In [1]:
# Importa librerías y carpetas con funciones
import sys
import os
import yaml
import pandas as pd
import glob
# Define the root and script directory
folder_root = os.getcwd()  # Get current directory (where Orquestación.ipynb is)
script_folder = os.path.join(folder_root, "Scripts")  # Path to 'Scripts'
# Ensure the script folder is added to sys.path
if script_folder not in sys.path:
    sys.path.append(script_folder)
contratos_library_scripts = os.path.join(script_folder, "Libreria_contratos")
if contratos_library_scripts not in sys.path:
    sys.path.append(contratos_library_scripts)

sanciones_folder = os.path.join(script_folder, "Libreria_SancionesIMSSB")
if sanciones_folder not in sys.path:
    sys.path.append(sanciones_folder)

In [2]:
# Cargar librerías internas
from folders_files_open import create_directory_if_not_exists
from STEP_C_PDFhandling import STEP_C_read_PDF_from_source
from dataframes_generation import create_dataframe #(extension, dataframe_name, columns, output_folder)
# Generador de carpetas
from folders_files_open import create_directory_if_not_exists
# Administración de contratos:
from administracion_de_contratos import administracion_de_contratos
from sanciones_IMSSB_processing import print_columns

In [3]:
#Cargar los folders requeridos. 
working_folder = desagregadas_folder = os.path.join(folder_root, "Implementación")
desagregadas_folder = os.path.join(folder_root, "Implementación", "Desagregadas")
create_directory_if_not_exists(working_folder)
create_directory_if_not_exists(desagregadas_folder)

	Folder Implementación encontrado.
	Folder Desagregadas encontrado.


# Procesa inputs
Las siguiente será la estructura de la carpeta con la información.

### Procesa archivos de desagregada raw

In [4]:
# Seleccionar el archivo XLSX de la carpeta para procesar.
XLSX_file_list = glob.glob(os.path.join(desagregadas_folder, "*.xlsx"))
XLSX_file_list.sort()
print(f"Found {len(XLSX_file_list)} files:")
for f in XLSX_file_list:
    print(f"\t {os.path.basename(f)}")

# prompt until a valid filename is entered
while True:
    XLSX_input = input("Ingresa el excel de la demanda desagregada:")
    # compare against basenames
    basenames = [os.path.basename(f) for f in XLSX_file_list]
    if XLSX_input in basenames:
        selected_path = os.path.join(desagregadas_folder, XLSX_input)
        # Using os.path
        parts    = os.path.normpath(selected_path).split(os.sep)
        folder1, folder2, fname = parts[-3], parts[-2], parts[-1]
        print(f"✅ Archivo seleccionado: {folder1}/{folder2}/{fname}")
        break
    else:
        print("❌ Selección no válida, elije un archivo de la lista. ")


Found 3 files:
	 Demanda desagregada global.xlsx
	 E115 Raw Demanda desagregada global.xlsx
	 desagregada.xlsx


Ingresa el excel de la demanda desagregada: E115 Raw Demanda desagregada global.xlsx


✅ Archivo seleccionado: Implementación/Desagregadas/E115 Raw Demanda desagregada global.xlsx


In [5]:
# Selecciona la hoja e imprime las columnas
# 1. load the workbook and list sheets
xls = pd.ExcelFile(selected_path)
sheets = xls.sheet_names
print("Available sheets:")
for name in sheets:
    print(" •", name)

# 2. prompt until a valid sheet is chosen
while True:
    selected_sheet = input("Select a sheet by name: ")
    if selected_sheet in sheets:
        break
    print("❌ Invalid sheet. Please choose one from the list above.")

# 3. read that sheet and print file path + its columns
df_input_xlsx = pd.read_excel(selected_path, sheet_name=selected_sheet)
print(f"✅ {XLSX_input} — Columnas:", )
print_columns(list(df_input_xlsx.columns))

Available sheets:
 • 100%
 • 40%
 • Propofol
 • Global
 • Sheet2
 • Sheet1


Select a sheet by name:  Global


✅ E115 Raw Demanda desagregada global.xlsx — Columnas:
INSTITUCION                AÑO                        CANT MAX ESEOTRES
PROV. 2  PROVEEDOR 2                
INSTITUCION HOMOLOGADA     FTE FMTO                   PRECIO ESEOTRES            PROCED.                    
UNIDAD                     CLAVE ADJ                  Total                      
ENTIDAD                    CANT MIN ESEOTRES          Descripción                


In [6]:
# Genera un dataframe con columnas adecuadas
from PROCESA_DESAGREGADA_input import standarized_dataframe_generation
print_columns(list(df_input_xlsx.columns), n_cols=3)
standarized_base_columns = ["Institución", "Procedimiento", "Clave", "Descripción", "Precio", "Piezas"]
raw_input_excel_columns = list(df_input_xlsx.columns)

df_clean = standarized_dataframe_generation(df_input_xlsx, standarized_base_columns)


INSTITUCION                FTE FMTO                   Total                      
INSTITUCION HOMOLOGADA     CLAVE ADJ                  Descripción                
UNIDAD                     CANT MIN ESEOTRES          PROVEEDOR 2                
ENTIDAD                    CANT MAX ESEOTRES
PROV. 2  PROCED.                    
AÑO                        PRECIO ESEOTRES            
Raw columns available: ['institucion', 'institucion homologada', 'unidad', 'entidad', 'año', 'fte fmto', 'clave adj', 'cant min eseotres', 'cant max eseotres prov 2', 'precio eseotres', 'total', 'descripción', 'proveedor 2', 'proced']


→ Which raw column maps to 'Institución'?  institucion homologada
→ Which raw column maps to 'Procedimiento'?  proced
→ Which raw column maps to 'Clave'?  clave adj
→ Which raw column maps to 'Descripción'?  descripción
→ Which raw column maps to 'Precio'?  precio eseotres
→ Which raw column maps to 'Piezas'?  cant max eseotres prov 2


In [9]:
print(df_clean.columns)
print(df_clean.info())
df_grouped = (
    df_clean
    .groupby(['Institución','Clave'], as_index=False)
    .agg(
        Piezas        = ('Piezas',       'sum'),
        Procedimiento = ('Procedimiento','first'),
        Descripción   = ('Descripción',  'first'),
        Precio        = ('Precio',       'first')
    )
)
print(df_grouped.columns)
print(df_grouped.info())
print(df_grouped.head(20))
#print(df_grouped['Procedimiento'].unique())

Index(['Institución', 'Procedimiento', 'Clave', 'Descripción', 'Precio',
       'Piezas'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618 entries, 0 to 617
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Institución    618 non-null    object
 1   Procedimiento  618 non-null    object
 2   Clave          618 non-null    object
 3   Descripción    618 non-null    object
 4   Precio         618 non-null    int64 
 5   Piezas         618 non-null    int64 
dtypes: int64(2), object(4)
memory usage: 29.1+ KB
None
Index(['Institución', 'Clave', 'Piezas', 'Procedimiento', 'Descripción',
       'Precio'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Institución    104 non-null    object
 1   Clave          104 non-null    o

## Procesa y elije demanda desagregada

In [40]:
# Carga la demanda desagregada que aplique


In [ ]:
#Genera la estructura de carpetas. 
create_directory_if_not_exists(bases_folder)
create_directory_if_not_exists(archivos_internos)
create_directory_if_not_exists(data_warehouse)
create_directory_if_not_exists(organizador_archivos)
create_directory_if_not_exists(sai_archivos)
create_directory_if_not_exists(camunda_archivos)
create_directory_if_not_exists(pisp_archivos)
create_directory_if_not_exists(sagi_archivos)
create_directory_if_not_exists(libreria_contratos)

## Generador de dataframes

In [ ]:
#Carga las columnas del archivo YAML
yaml_file_path = os.path.join(folder_root, "df_headers.yaml")
with open(yaml_file_path, "r") as f:
    columns_data = yaml.safe_load(f)

columnas_desagregada = columns_data["Desagregada_columns"]
df_desagregada = create_dataframe("xlsx", "desagregada", columns_data["Desagregada_columns"], data_warehouse)
#print(df_desagregada.head())

# Administrador de contratos
Aquí empieza todo, la demanda desagregada nos da una lista de instituciones a las que debemos de visitar. 

## Capturar y Exportar base de contratos
1) Captura de contrato: te va haciendo preguntas hasta que tienes un PDF con la leyenda que necesitamos en su cuerpo.
2) Extrae la información de los diccionarios del PDF y genera un archivo .pickle con la información.
3) Exporta pickle a excel para consulta de los usuarios.

In [ ]:
# Capturar un nuevo contato
administracion_de_contratos(data_warehouse, working_folder, libreria_contratos)

In [ ]:
STEP_C_read_PDF_from_source(libreria_contratos)